In [1]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 19.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install --upgrade xgboost


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [24]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Function to load and flatten your data (landmarks, motion, velocity)
def load_data(folder_landmarks, folder_motion, folder_velocity, class_label):
    X = []
    y = []
    for fname in os.listdir(folder_landmarks):
        if not fname.endswith('_landmarks.npy'):
            continue

        landmarks = np.load(os.path.join(folder_landmarks, fname))  # shape (T, 71, 2)
        motion = np.load(os.path.join(folder_motion, fname.replace('_landmarks.npy', '_motion.npy')))
        velocity = np.load(os.path.join(folder_velocity, fname.replace('_landmarks.npy', '_velocity.npy')))

        # Flatten features: concat landmarks + motion + velocity for each frame, then flatten frames
        features = np.concatenate([landmarks, motion, velocity], axis=2)  # (T, 71, 6)
        features = features.reshape(features.shape[0], -1)  # (T, 426)

        # Average frames to get a fixed-length feature vector per video
        features_avg = np.mean(features, axis=0)  # (426,)

        X.append(features_avg)
        y.append(class_label)
    return X, y

# Load data
classes = ['Non-suspicious', 'Suspicious']
X, y = [], []

for i, cls in enumerate(classes):
    Xc, yc = load_data(
        f'transformed/landmarks.npy/Train/{cls}',
        f'transformed/motion.npy/Train/{cls}',
        f'transformed/velocity.npy/Train/{cls}',
        i
    )
    X.extend(Xc)
    y.extend(yc)

X = np.array(X)
y = np.array(y)

# Split train/validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize XGBoost classifier
model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    n_estimators=100,
    max_depth=10,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8
)

# Train
model.fit(
    X_train,
    y_train,
    # eval_set=[(X_val, y_val)],
    # eval_metric='logloss',
    # early_stopping_rounds=10,
    # verbose=True
)


# Evaluate
y_pred = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred, target_names=classes))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred))


/mnt/c/Users/Raymond/Desktop/3D-CNN/extract/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [19:28:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.8085106382978723

Classification Report:
                 precision    recall  f1-score   support

Non-suspicious       0.83      0.79      0.81        24
    Suspicious       0.79      0.83      0.81        23

      accuracy                           0.81        47
     macro avg       0.81      0.81      0.81        47
  weighted avg       0.81      0.81      0.81        47


Confusion Matrix:
 [[19  5]
 [ 4 19]]
